In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt

data = pd.read_csv('JSE_clean_truncated.csv')

# Normalize the data (MinMax scaling)
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.values)

# Sliding window creation
def create_sliding_window(data, input_window, output_window):
    x, y = [], []
    for i in range(len(data) - input_window - output_window):
        x.append(data[i:(i + input_window)])
        y.append(data[(i + input_window):(i + input_window + output_window)])
    return np.array(x), np.array(y)

# MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation on the output layer
        return x

# Training and evaluation loop with logging and plotting
def train_and_evaluate_model(hyperparameters, run_id, share_name):
    # Extract hyperparameters
    input_window = hyperparameters['input_window']
    output_window = hyperparameters['output_window']
    hidden_nodes = hyperparameters['hidden_nodes']
    learning_rate = hyperparameters['learning_rate']
    batch_size = hyperparameters['batch_size']
    epochs = hyperparameters['epochs']

    # Create sliding window
    x, y = create_sliding_window(data_normalized, input_window, output_window)

    # Split data (7:2:1 train/val/test)
    num_samples = x.shape[0]
    num_train = int(0.7 * num_samples)
    num_val = int(0.2 * num_samples)
    num_test = num_samples - num_train - num_val

    x_train, y_train = x[:num_train], y[:num_train]
    x_val, y_val = x[num_train:num_train + num_val], y[num_train:num_train + num_val]
    x_test, y_test = x[num_train + num_val:], y[num_train + num_val:]

    # Convert to PyTorch tensors
    train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    valid_dataset = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    input_dim = input_window * data.shape[1]  # Adjust input size based on number of stocks (columns)
    output_dim = output_window * data.shape[1]  # Adjust for output dimensions

    model = MLP(input_size=input_dim, hidden_size=hidden_nodes, output_size=output_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Lists to store losses for plotting
    train_losses = []
    valid_losses = []

    # Start training and evaluation
    log_lines = []
    start_time = time.time()

    for epoch in range(epochs):
        train_loss = 0
        model.train()
        for x, y in train_loader:
            x = x.view(x.size(0), -1)
            y = y.view(y.size(0), -1)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        valid_loss = 0
        model.eval()
        with torch.no_grad():
            for x, y in valid_loader:
                x = x.view(x.size(0), -1)
                y = y.view(y.size(0), -1)
                output = model(x)
                loss = criterion(output, y)
                valid_loss += loss.item()
        avg_valid_loss = valid_loss / len(valid_loader)
        valid_losses.append(avg_valid_loss)
        log_lines.append(f'Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_valid_loss:.4f}')

    # Testing
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x = x.view(x.size(0), -1)
            y = y.view(y.size(0), -1)
            output = model(x)
            loss = criterion(output, y)
            test_loss += loss.item()
    avg_test_loss = test_loss / len(test_loader)

    # Log the results
    elapsed_time = time.time() - start_time
    log_lines.append(f'Test Loss: {avg_test_loss:.4f}')
    log_lines.append(f'Training Time: {elapsed_time:.2f} seconds')
    log_lines.append(f'Hyperparameters: {hyperparameters}')
    log_lines.append('---' * 10)

    # Save logs to file
    with open(f'logss_{share_name}.txt', 'a') as f:
        for line in log_lines:
            f.write(line + '\n')

    # Print final metrics for easy comparison
    print(f"Run {run_id} ({share_name}) - Final Test Loss: {avg_test_loss:.4f}, Training Time: {elapsed_time:.2f}s")

    # Plot training and validation losses
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs + 1), valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss (Run {run_id}, {share_name})')
    plt.legend()
    plt.savefig(f'loss2_plot_run_{run_id}_{share_name}.png')
    plt.close()

# Define hyperparameter configurations to test
hyperparameter_configurations = [
    {'input_window': 30, 'output_window': 1, 'hidden_nodes': 32, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 50},
    {'input_window': 60, 'output_window': 1, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 100},
    {'input_window': 30, 'output_window': 2, 'hidden_nodes': 128, 'learning_rate': 0.0005, 'batch_size': 16, 'epochs': 100},
    # Add more configurations if needed
]

# Get the list of share names from the dataset for iteration
share_names = data.columns

# Run experiments with each configuration on each share
for share_name in share_names:
    for i, config in enumerate(hyperparameter_configurations, 1):
        train_and_evaluate_model(config, run_id=i, share_name=share_name)

print("All experiments completed, results logged in log files, and performance graphs saved.")


Run 1 (ASPEN) - Final Test Loss: 0.0590, Training Time: 7.17s
Run 2 (ASPEN) - Final Test Loss: 0.0767, Training Time: 14.36s
Run 3 (ASPEN) - Final Test Loss: 0.0430, Training Time: 39.16s
Run 1 (CAPITEC) - Final Test Loss: 0.0672, Training Time: 6.25s
Run 2 (CAPITEC) - Final Test Loss: 0.0514, Training Time: 16.92s
Run 3 (CAPITEC) - Final Test Loss: 0.0478, Training Time: 43.50s
Run 1 (IMPLATS) - Final Test Loss: 0.0493, Training Time: 7.06s
Run 2 (IMPLATS) - Final Test Loss: 0.0525, Training Time: 15.61s
Run 3 (IMPLATS) - Final Test Loss: 0.0491, Training Time: 41.99s
Run 1 (GROWPNT) - Final Test Loss: 0.0652, Training Time: 7.25s
Run 2 (GROWPNT) - Final Test Loss: 0.0766, Training Time: 14.89s
Run 3 (GROWPNT) - Final Test Loss: 0.0537, Training Time: 40.99s
Run 1 (NORTHAM) - Final Test Loss: 0.0692, Training Time: 6.86s
Run 2 (NORTHAM) - Final Test Loss: 0.0477, Training Time: 14.92s
Run 3 (NORTHAM) - Final Test Loss: 0.0583, Training Time: 42.85s
Run 1 (ANGGOLD) - Final Test Loss: 0

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt
import csv

# Load dataset
data = pd.read_csv('JSE_clean_truncated.csv')

# Normalize the data (MinMax scaling)
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.values)

# Create sliding window
def create_sliding_window(data, input_window, output_window):
    x, y = [], []
    for i in range(len(data) - input_window - output_window):
        x.append(data[i:(i + input_window)])
        y.append(data[(i + input_window):(i + input_window + output_window)])
    return np.array(x), np.array(y)

# MLP Model with Batch Normalization and Dropout
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(0.3)  # Dropout rate
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))  # BatchNorm + ReLU
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # Output layer
        return x

# Huber loss function
def get_loss_function():
    return nn.HuberLoss()

# Manual metrics calculation
def calculate_metrics(y_true, y_pred):
    epsilon = 1e-10  # A small constant to avoid division by zero
    mape = torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    mae = torch.mean(torch.abs(y_true - y_pred))
    rmse = torch.sqrt(torch.mean((y_true - y_pred) ** 2))

    return {
        'MAPE': mape.item(),
        'MAE': mae.item(),
        'RMSE': rmse.item(),
    }

# Training and evaluation loop with logging and plotting
def train_and_evaluate_model(hyperparameters, run_id, csv_writer):
    # Extract hyperparameters
    input_window = hyperparameters['input_window']
    output_window = hyperparameters['output_window']
    hidden_nodes = hyperparameters['hidden_nodes']
    learning_rate = hyperparameters['learning_rate']
    batch_size = hyperparameters['batch_size']
    epochs = hyperparameters['epochs']

    # Create sliding window
    x, y = create_sliding_window(data_normalized, input_window, output_window)

    # Split data (60% train, 20% validation, 20% test to preserve temporal order)
    num_samples = x.shape[0]
    num_train = int(0.6 * num_samples)
    num_val = int(0.2 * num_samples)
    num_test = num_samples - num_train - num_val

    x_train, y_train = x[:num_train], y[:num_train]
    x_val, y_val = x[num_train:num_train + num_val], y[num_train:num_train + num_val]
    x_test, y_test = x[num_train + num_val:], y[num_train + num_val:]

    # Convert to PyTorch tensors
    train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    valid_dataset = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    # Initialize the model
    input_dim = input_window * data.shape[1]
    output_dim = output_window * data.shape[1]

    model = MLP(input_size=input_dim, hidden_size=hidden_nodes, output_size=output_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    criterion = get_loss_function()

    # Early stopping variables
    best_val_loss = float('inf')
    patience, wait = 10, 0

    # Lists to store losses for plotting
    train_losses = []
    valid_losses = []
    metrics_store = []  # Store metrics for test evaluation

    # Start training and evaluation
    start_time = time.time()

    for epoch in range(epochs):
        train_loss = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            y_batch = y_batch.view(y_batch.size(0), -1)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        valid_loss = 0
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_loader:
                x_batch = x_batch.view(x_batch.size(0), -1)
                y_batch = y_batch.view(y_batch.size(0), -1)
                output = model(x_batch)
                loss = criterion(output, y_batch)
                valid_loss += loss.item()
        avg_valid_loss = valid_loss / len(valid_loader)
        valid_losses.append(avg_valid_loss)

        # Update learning rate scheduler
        scheduler.step(avg_valid_loss)

        # Early stopping
        if avg_valid_loss < best_val_loss:
            best_val_loss = avg_valid_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Testing
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            y_batch = y_batch.view(y_batch.size(0), -1)
            output = model(x_batch)
            loss = criterion(output, y_batch)
            test_loss += loss.item()
            # Calculate performance metrics
            metrics = calculate_metrics(y_batch, output)
            metrics_store.append(metrics)

    elapsed_time = time.time() - start_time

    # Write to CSV file
    for metrics in metrics_store:
        csv_writer.writerow([run_id, input_window, output_window, test_loss, metrics['MAPE'], metrics['MAE'], metrics['RMSE'], elapsed_time])

    # Generate plots
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(valid_losses) + 1), valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss (Run {run_id}, Input Window: {input_window}, Horizon: {output_window})')
    plt.legend()
    plt.savefig(f'loss_plot_run_{run_id}.png')
    plt.close()




In [3]:
# Define hyperparameter configurations for all horizons and input windows
hyperparameter_configurations = [
    # Input Window 30
    {'input_window': 30, 'output_window': 1, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 30, 'output_window': 2, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 30, 'output_window': 5, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 30, 'output_window': 10, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 30, 'output_window': 30, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},

    # Input Window 60
    {'input_window': 60, 'output_window': 1, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 60, 'output_window': 2, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 60, 'output_window': 5, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 60, 'output_window': 10, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 60, 'output_window': 30, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},

    # Input Window 120
    {'input_window': 120, 'output_window': 1, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 120, 'output_window': 2, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 120, 'output_window': 5, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 120, 'output_window': 10, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
    {'input_window': 120, 'output_window': 30, 'hidden_nodes': 64, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100},
]
# Run experiments with each configuration and save to CSV
with open('performance_log.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Run', 'Input Window', 'Horizon', 'Test Loss', 'MAPE', 'MAE', 'RMSE', 'Training Time'])

    for i, config in enumerate(hyperparameter_configurations, 1):
        train_and_evaluate_model(config, run_id=i, csv_writer=writer)

print("All experiments completed, results logged in performance_log.csv, and performance graphs saved.")


Early stopping at epoch 22
Early stopping at epoch 19
Early stopping at epoch 41
Early stopping at epoch 19
Early stopping at epoch 11
Early stopping at epoch 23
Early stopping at epoch 25
Early stopping at epoch 24
Early stopping at epoch 28
Early stopping at epoch 36
Early stopping at epoch 26
Early stopping at epoch 37
Early stopping at epoch 39
Early stopping at epoch 25
Early stopping at epoch 41
All experiments completed, results logged in performance_log.csv, and performance graphs saved.


In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt
import csv

# Load dataset
data = pd.read_csv('JSE_clean_truncated.csv')

# Normalize the data (MinMax scaling)
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.values)

# Create sliding window
def create_sliding_window(data, input_window, output_window):
    x, y = [], []
    for i in range(len(data) - input_window - output_window):
        x.append(data[i:(i + input_window)])
        y.append(data[(i + input_window):(i + input_window + output_window)])
    return np.array(x), np.array(y)

# MLP Model with Batch Normalization and Dropout
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(0.3)  # Dropout rate
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))  # BatchNorm + ReLU
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # Output layer
        return x

# Huber loss function
def get_loss_function():
    return nn.HuberLoss()

# Manual metrics calculation
def calculate_metrics(y_true, y_pred):
    epsilon = 1e-10  # A small constant to avoid division by zero
    mape = torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    mae = torch.mean(torch.abs(y_true - y_pred))
    rmse = torch.sqrt(torch.mean((y_true - y_pred) ** 2))

    return {
        'MAPE': mape.item(),
        'MAE': mae.item(),
        'RMSE': rmse.item(),
    }

# Training and evaluation loop with logging and plotting
def train_and_evaluate_model(hyperparameters, run_id, csv_writer):
    # Extract hyperparameters
    input_window = hyperparameters['input_window']
    output_window = hyperparameters['output_window']
    hidden_nodes = hyperparameters['hidden_nodes']
    learning_rate = hyperparameters['learning_rate']
    batch_size = hyperparameters['batch_size']
    epochs = hyperparameters['epochs']
    weight_decay = hyperparameters.get('weight_decay', 0.0)  # Add weight decay

    # Sliding window creation and data split remains the same



    # Create sliding window
    x, y = create_sliding_window(data_normalized, input_window, output_window)

    # Split data (60% train, 20% validation, 20% test to preserve temporal order)
    num_samples = x.shape[0]
    num_train = int(0.6 * num_samples)
    num_val = int(0.2 * num_samples)
    num_test = num_samples - num_train - num_val

    x_train, y_train = x[:num_train], y[:num_train]
    x_val, y_val = x[num_train:num_train + num_val], y[num_train:num_train + num_val]
    x_test, y_test = x[num_train + num_val:], y[num_train + num_val:]

    # Convert to PyTorch tensors
    train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    valid_dataset = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    # Initialize the model
    input_dim = input_window * data.shape[1]
    output_dim = output_window * data.shape[1]

    model = MLP(input_size=input_dim, hidden_size=hidden_nodes, output_size=output_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  # Apply weight decay
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    criterion = get_loss_function()

    # Early stopping variables
    best_val_loss = float('inf')
    patience, wait = 10, 0

    # Lists to store losses for plotting
    train_losses = []
    valid_losses = []
    metrics_store = []  # Store metrics for test evaluation

    # Start training and evaluation
    start_time = time.time()

    for epoch in range(epochs):
        train_loss = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            y_batch = y_batch.view(y_batch.size(0), -1)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        valid_loss = 0
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_loader:
                x_batch = x_batch.view(x_batch.size(0), -1)
                y_batch = y_batch.view(y_batch.size(0), -1)
                output = model(x_batch)
                loss = criterion(output, y_batch)
                valid_loss += loss.item()
        avg_valid_loss = valid_loss / len(valid_loader)
        valid_losses.append(avg_valid_loss)

        # Update learning rate scheduler
        scheduler.step(avg_valid_loss)

        # Early stopping
        if avg_valid_loss < best_val_loss:
            best_val_loss = avg_valid_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Testing
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            y_batch = y_batch.view(y_batch.size(0), -1)
            output = model(x_batch)
            loss = criterion(output, y_batch)
            test_loss += loss.item()
            # Calculate performance metrics
            metrics = calculate_metrics(y_batch, output)
            metrics_store.append(metrics)

    elapsed_time = time.time() - start_time

    # Write to CSV file
    for metrics in metrics_store:
        csv_writer.writerow([run_id, input_window, output_window, test_loss, metrics['MAPE'], metrics['MAE'], metrics['RMSE'], elapsed_time])

    # Generate plots
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(valid_losses) + 1), valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss (Run {run_id}, Input Window: {input_window}, Horizon: {output_window})')
    plt.legend()
    plt.savefig(f'loss_plot_run_{run_id}.png')
    plt.close()




In [5]:
hyperparameter_configurations = [
    {'input_window': 30, 'output_window': 1, 'hidden_nodes': 64, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 50, 'weight_decay': 0.01},
    {'input_window': 60, 'output_window': 2, 'hidden_nodes': 128, 'learning_rate': 0.0001, 'batch_size': 64, 'epochs': 100, 'weight_decay': 0.001},
    # Add other configurations to explore
]

# Run experiments with each configuration and save to CSV
with open('performance_log.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Run', 'Input Window', 'Horizon', 'Test Loss', 'MAPE', 'MAE', 'RMSE', 'Training Time'])

    for i, config in enumerate(hyperparameter_configurations, 1):
        train_and_evaluate_model(config, run_id=i, csv_writer=writer)

print("All experiments completed, results logged in performance_log.csv, and performance graphs saved.")


Early stopping at epoch 15
Early stopping at epoch 23
All experiments completed, results logged in performance_log.csv, and performance graphs saved.
